In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Dados base
CTD_base = 217062.50
margem_base = 0.30
GBAC_base = 20493000
percentual_bonus = 0.05
teto_bonus_anual = 1_000_000
custo_gestao_bonus_anual = 10_000

# Função de cálculo ajustada
def calcular_resultados(custo_var, margem_var, gbac_var):
    CTD = CTD_base * (1 + custo_var)
    margem = margem_base * (1 + margem_var)

    receita_desenvolvimento = CTD * (1 + margem)
    lucro_desenvolvimento = receita_desenvolvimento - CTD

    GBAC = GBAC_base * (1 + gbac_var)
    bonus_anual_bruto = GBAC * percentual_bonus
    bonus_anual_efetivo = min(bonus_anual_bruto, teto_bonus_anual)

    lucro_bonus_5anos = (bonus_anual_efetivo - custo_gestao_bonus_anual) * 5
    lucro_total_consultoria = lucro_desenvolvimento + lucro_bonus_5anos

    gasto_total_cliente = receita_desenvolvimento + bonus_anual_efetivo * 5
    ganho_liquido_cliente = (GBAC * 5) - gasto_total_cliente

    return lucro_total_consultoria, ganho_liquido_cliente, bonus_anual_efetivo

# Faixas de variação
custos = {'Base': 0, '+50%': 0.5, '+100%': 1.0}
margens = {'Base (30%)': 0, '+50% (45%)': 0.5, '+100% (60%)': 1.0}
ganhos_cliente = {'Base': 0, '-20%': -0.2, '-50%': -0.5}

# Criar cenários
def gerar_cenarios(custo, margem):
    dados = []
    for gb_label, gb_var in ganhos_cliente.items():
        lucro_consultoria, ganho_cliente, bonus_efetivo = calcular_resultados(custos[custo], margens[margem], gb_var)
        dados.append({
            'Ganho Cliente': gb_label,
            'Lucro Consultoria': lucro_consultoria,
            'Ganho Líquido Cliente': ganho_cliente,
            'Bônus Anual Efetivo': bonus_efetivo
        })
    return pd.DataFrame(dados)

# Função de plotagem com gráficos separados
from plotly.subplots import make_subplots

def plotar(custo, margem):
    df = gerar_cenarios(custo, margem)

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=("Lucro da Consultoria (5 anos)", "Ganho Líquido do Cliente (5 anos)", "Bônus Anual Efetivo"),
        vertical_spacing=0.15
    )

    fig.add_trace(go.Bar(
        x=df['Ganho Cliente'],
        y=df['Lucro Consultoria'],
        name='Lucro Consultoria',
        marker_color='royalblue',
        text=df['Lucro Consultoria'].apply(lambda x: f'R$ {x:,.0f}'),
        textposition='outside'
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=df['Ganho Cliente'],
        y=df['Ganho Líquido Cliente'],
        name='Ganho Cliente',
        marker_color='seagreen',
        text=df['Ganho Líquido Cliente'].apply(lambda x: f'R$ {x:,.0f}'),
        textposition='outside'
    ), row=2, col=1)

    fig.add_trace(go.Bar(
        x=df['Ganho Cliente'],
        y=df['Bônus Anual Efetivo'],
        name='Bônus Anual Efetivo',
        marker_color='darkorange',
        text=df['Bônus Anual Efetivo'].apply(lambda x: f'R$ {x:,.0f}'),
        textposition='outside'
    ), row=3, col=1)

    fig.update_layout(
        height=1000,
        title_text=f"Simulação Financeira Separada por Métrica (Custo: {custo}, Margem: {margem})",
        showlegend=False,
        uniformtext_minsize=8,
        uniformtext_mode='hide'
    )

    fig.update_yaxes(title_text="R$", row=1, col=1)
    fig.update_yaxes(title_text="R$", row=2, col=1)
    fig.update_yaxes(title_text="R$", row=3, col=1)

    fig.show()

# Widgets
custo_widget = widgets.Dropdown(options=custos.keys(), description='Custo:')
margem_widget = widgets.Dropdown(options=margens.keys(), description='Margem:')

# Exibir interface
interface = widgets.interactive(plotar, custo=custo_widget, margem=margem_widget)
display(interface)

interactive(children=(Dropdown(description='Custo:', options=('Base', '+50%', '+100%'), value='Base'), Dropdow…